In [45]:
import pandas as pd
import blackjack_utils.game_config as gc
import blackjack_utils.shoe as shoe
import blackjack_utils.card as card
import random
import blackjack_utils.deck as deck
from blackjack_utils.utils import simulate_hand, build_combos, determine_best_action

DEALER_HIT_SOFT_17 = True
DOUBLE_AFTER_SPLIT = True
SURRENDER_ALLOWED = True
BLACKJACK_PAYS = 1.5
DECKS_IN_SHOE = 6
SAMPLE_SIZE = 3_000_000 # should run for about 2 hours

In [46]:
game_config = gc.GameConfig(DECKS_IN_SHOE, DEALER_HIT_SOFT_17, DOUBLE_AFTER_SPLIT, SURRENDER_ALLOWED, BLACKJACK_PAYS)

In [47]:
my_ev_action_table = pd.read_csv('data2/double_after_splitting_hit_soft_17_odds_1.4m.csv', dtype={'player_total': str, 'dealer_card_up': str})

In [48]:
my_ev_action_table.head()

,double,hit,stand,player_total,dealer_card_up,best_action,split
0,-1.708570,-0.854285,0.633407,20,2,stand,NaN
1,-1.707650,-0.853825,0.643708,20,3,stand,NaN
2,-1.707966,-0.853983,0.653404,20,4,stand,NaN
3,-1.707286,-0.853643,0.669179,20,5,stand,NaN
4,-1.706294,-0.853147,0.677230,20,6,stand,NaN


In [49]:
profit_loss = 0.0
for i in range(SAMPLE_SIZE // 10):
    deck = shoe.Shoe(6)
    deck.shuffle()
    for j in range(10):
        player_cards = [deck.draw(), deck.draw()]
        dealer_card_up = deck.draw()
        profit_loss += simulate_hand(game_config, player_cards, dealer_card_up, deck, my_ev_action_table)
        

In [50]:
profit_loss / SAMPLE_SIZE

-0.0017473333333333334

In [51]:
consensus_ev_actions = pd.read_csv('double_after_splitting_hit_soft_17_odds_1.4m-adjusted_to_consensus.csv', dtype={'dealer_card_up': str, 'player_total': str})

In [52]:
consensus_profit_loss = 0.0
for i in range(SAMPLE_SIZE // 10):
    deck = shoe.Shoe(6)
    deck.shuffle()
    for j in range(10):
        player_cards = [deck.draw(), deck.draw()]
        dealer_card_up = deck.draw()
        consensus_profit_loss += simulate_hand(game_config, player_cards, dealer_card_up, deck, consensus_ev_actions)

In [53]:
consensus_profit_loss / SAMPLE_SIZE

-0.0011571666666666666

With 3 million samples, my version had a per-hand EV of -0.0017473333333333334, while the consensus version had a per-hand EV of -0.0011571666666666666.

In [54]:
profit_loss

-5242.0

In [56]:
1_500_000 + (profit_loss / 2)

1497379.0

In [57]:
1_500_000 + (consensus_profit_loss / 2)

1498264.25

In [59]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

#not perfectly true because of pushes, splits, and blackjacks, but we can back into a "number of successes" and do a proportions z-test
count = np.array([1497379, 1498264])  # Number of successes in each group
nobs = np.array([3_000_000, 3_000_000]) # Number of observations (trials) in each group

# Perform the two-sample proportions z-test
z_statistic, p_value = proportions_ztest(count, nobs)

print(f"Z-statistic: {z_statistic:.4f}")
print(f"P-value: {p_value:.4f}")

Z-statistic: -0.7226
P-value: 0.4699


Z-statistic: -0.7226

P-value: 0.4699

Not at all significant